In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from numpy import concatenate
from keras.models import load_model

#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

print(tf.__version__)

Couldn't import dot_parser, loading of dot files will not be possible.
2.2.0


Using TensorFlow backend.


In [2]:
df=pd.read_csv("data_sin_varnula.csv")
#df = df.drop(['petroleo(USD)','nasdaq(USD)'], axis=1)
df

,date,dow(USD),euro_stoxx50(USD),nasdaq(USD),nikkei(USD),oro(USD),petroleo(USD),sp_500(USD),BCHAIN-TOTBC,BCHAIN-MIREV (USD),BCHAIN-HRATE,BCHAIN-NTRBL,BCHAIN-NADDU,BCHAIN-MKPRU (USD)
0,2015-01-15,43.800,3112.370,45.670,16872.950,1380.00,48.600,2013.750,13727150.00,7.614220e+05,3.038322e+05,824.000000,212403.0,218.11
1,2015-01-16,43.260,3147.530,45.730,16812.960,1404.90,46.350,1992.250,13730725.00,7.680090e+05,3.256906e+05,731.000000,219873.0,205.35
2,2015-01-17,43.260,3147.530,45.730,16812.960,1342.95,47.650,1992.250,13734300.00,6.775014e+05,2.994605e+05,633.000000,254267.0,197.12
3,2015-01-18,43.260,3178.075,45.730,16906.870,1281.00,48.950,1992.250,13738025.00,7.891797e+05,3.256906e+05,703.000000,223392.0,211.18
4,2015-01-19,44.130,3208.620,46.165,17000.780,1279.40,48.840,2006.505,13741900.00,8.261971e+05,3.388057e+05,740.000000,197559.0,212.39
5,2015-01-20,45.000,3229.340,46.600,17071.650,1380.00,48.690,2020.760,13744900.00,6.419519e+05,2.623012e+05,653.000000,194860.0,212.99
6,2015-01-21,43.950,3253.450,45.740,17308.720,1391.10,46.690,2020.190,13748325.00,7.398576e+05,2.994605e+05,830.000000,197159.0,215.20
7,2015-01-22,44.980,3270.400,45.690,17306.640,1425.20,47.350,2034.300,13752075.00,8.801657e+05,3.278765e+05,683.000000,188687.0,233.90
8,2015-01-23,45.150,3345.450,46.460,17520.630,1417.60,46.630,2062.980,13755175.00,7.254224e+05,2.710446e+05,565.000000,179966.0,233.03
9,2015-01-24,45.150,3345.450,46.460,17520.630,1417.60,46.630,2062.980,13758200.00,7.429717e+05,2.644870e+05,520.000000,172854.0,244.64


In [3]:
df.iloc[1082]

date                   2018-01-01
dow(USD)                    64.91
euro_stoxx50(USD)         3570.14
nasdaq(USD)                78.555
nikkei(USD)               23053.8
oro(USD)                   1305.8
petroleo(USD)               60.26
sp_500(USD)                2752.8
BCHAIN-TOTBC          1.67745e+07
BCHAIN-MIREV (USD)    3.54352e+07
BCHAIN-HRATE          1.51774e+07
BCHAIN-NTRBL              1781.73
BCHAIN-NADDU               678712
BCHAIN-MKPRU (USD)        14165.6
Name: 1082, dtype: object

In [4]:
df.shape

(2177, 14)

In [5]:
#plt.plot(df.iloc[1900:,0], df.iloc[1900:,-1])

In [6]:
from pandas import DataFrame
from pandas import concat

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg





In [7]:
ventana=2 # Se puede mover 
n_cols=14
values = df.iloc[1082:,1:].values# Desde el primer dia del 2018

scaler = MinMaxScaler(feature_range=(0, 10))
values = scaler.fit_transform(values)

data = series_to_supervised(values, ventana,1 )

In [8]:
data

,var1(t-2),var2(t-2),var3(t-2),var4(t-2),var5(t-2),var6(t-2),var7(t-2),var8(t-2),var9(t-2),var10(t-2),...,var4(t),var5(t),var6(t),var7(t),var8(t),var9(t),var10(t),var11(t),var12(t),var13(t)
2,10.000000,7.990043,0.564329,5.899993,1.157774,8.234642,3.166518,0.000000,7.279866,0.097602,...,4.851022,2.307516,8.377689,2.908997,0.022580,8.494152,0.180962,6.014824,7.261453,4.947446
3,10.000000,8.301338,0.836043,6.102299,2.756168,8.733644,3.602686,0.010772,6.544184,0.084018,...,5.102325,2.309811,8.766356,2.663743,0.033421,8.270977,0.090481,8.752944,8.426239,4.967355
4,10.000000,7.041817,0.942870,4.851022,2.307516,8.377689,2.908997,0.022580,8.494152,0.180962,...,5.353628,2.312106,9.155023,2.418488,0.045022,8.962597,0.161573,8.793144,10.000000,5.028709
5,10.000000,7.401883,1.485524,5.102325,2.309811,8.766356,2.663743,0.033421,8.270977,0.090481,...,5.055890,2.151463,8.989798,2.932845,0.054896,8.735349,0.000000,8.102124,7.766441,5.858045
6,10.000000,7.761949,2.028178,5.353628,2.312106,9.155023,2.418488,0.045022,8.962597,0.161573,...,5.290649,1.644291,9.065203,3.283938,0.066497,10.000000,0.161573,6.750968,7.775789,5.918973
7,10.000000,6.980126,2.745007,5.055890,2.151463,8.989798,2.932845,0.054896,8.735349,0.000000,...,5.525407,1.137120,9.140608,3.635030,0.077683,9.270757,0.122795,7.460180,8.056560,5.638550
8,10.000000,7.341951,2.665273,5.290649,1.644291,9.065203,3.283938,0.066497,10.000000,0.161573,...,6.222129,0.909925,9.410069,3.995203,0.089146,8.611994,0.148647,6.472850,7.868430,5.056659
9,10.000000,7.703776,2.585540,5.525407,1.137120,9.140608,3.635030,0.077683,9.270757,0.122795,...,6.918851,0.682731,9.679530,4.355376,0.100470,8.361294,0.135721,7.065962,8.179379,4.824982
10,10.000000,7.271331,2.183000,6.222129,0.909925,9.410069,3.995203,0.089146,8.611994,0.148647,...,4.855827,1.231211,8.746951,2.925170,0.111933,8.616299,0.148647,6.526612,7.613163,4.998070
11,10.000000,6.838887,1.780461,6.918851,0.682731,9.679530,4.355376,0.100470,8.361294,0.135721,...,4.855827,1.231211,8.746951,2.925170,0.125260,8.395229,0.323146,4.754967,6.874785,4.229696


In [9]:
entrenamiento=600
n_obs=ventana*n_cols

train=data.iloc[:entrenamiento,:]
test =data.iloc[entrenamiento:,:]

train_X, train_y = train.iloc[:, :n_obs].values, train.iloc[:, -1].values
test_X, test_y = test.iloc[:, :n_obs].values, test.iloc[:, -1].values

print(train_X.shape, train_y.shape)
print(test_X.shape, test_y.shape)

(600, 28) (600,)
(493, 28) (493,)


In [10]:
train_X = train_X.reshape((train_X.shape[0], ventana, n_cols))
test_X = test_X.reshape((test_X.shape[0], ventana, n_cols))

In [11]:
test_X.shape

(493, 2, 14)

In [12]:
tf.keras.backend.clear_session()

model = tf.keras.models.Sequential([
   tf.keras.layers.Conv1D(filters=32, kernel_size=5,
                       strides=1, padding="causal",
                       activation="relu",
                       input_shape=[train_X.shape[1],train_X.shape[2]]),
   tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(30,activation="relu"),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 50.0)# se puede mover entre 500 a unos 2000
])


model.compile(loss=tf.keras.losses.Huber(), optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6),metrics=["mae"])
history = model.fit(train_X,train_y,shuffle=False,validation_data=(test_X,test_y),epochs=450)# epocas se pueden mover, ver la grafica del trainning

Epoch 1/450
19/19 [==============================] - 3s 145ms/step - loss: 1.9277 - mae: 2.3886 - val_loss: 0.9721 - val_mae: 1.3760
Epoch 2/450
19/19 [==============================] - 0s 17ms/step - loss: 1.8498 - mae: 2.3134 - val_loss: 0.9343 - val_mae: 1.3304
Epoch 3/450
19/19 [==============================] - 0s 16ms/step - loss: 1.8163 - mae: 2.2762 - val_loss: 0.8990 - val_mae: 1.2876
Epoch 4/450
19/19 [==============================] - 0s 14ms/step - loss: 1.7074 - mae: 2.1705 - val_loss: 0.8648 - val_mae: 1.2459
Epoch 5/450
19/19 [==============================] - 0s 15ms/step - loss: 1.7274 - mae: 2.1823 - val_loss: 0.8339 - val_mae: 1.2076
Epoch 6/450
19/19 [==============================] - 0s 15ms/step - loss: 1.7687 - mae: 2.2294 - val_loss: 0.8052 - val_mae: 1.1714
Epoch 7/450
19/19 [==============================] - 0s 15ms/step - loss: 1.6434 - mae: 2.0975 - val_loss: 0.7776 - val_mae: 1.1366
Epoch 8/450
19/19 [==============================] - 0s 14ms/step - loss: 1

19/19 [==============================] - 0s 14ms/step - loss: 1.0452 - mae: 1.4824 - val_loss: 0.6443 - val_mae: 1.0555
Epoch 63/450
19/19 [==============================] - 0s 19ms/step - loss: 1.0323 - mae: 1.4580 - val_loss: 0.6463 - val_mae: 1.0579
Epoch 64/450
19/19 [==============================] - 0s 16ms/step - loss: 1.1311 - mae: 1.5619 - val_loss: 0.6479 - val_mae: 1.0598
Epoch 65/450
19/19 [==============================] - 0s 14ms/step - loss: 1.0133 - mae: 1.4408 - val_loss: 0.6499 - val_mae: 1.0621
Epoch 66/450
19/19 [==============================] - 0s 15ms/step - loss: 1.0513 - mae: 1.4743 - val_loss: 0.6505 - val_mae: 1.0628
Epoch 67/450
19/19 [==============================] - 0s 14ms/step - loss: 0.9335 - mae: 1.3513 - val_loss: 0.6514 - val_mae: 1.0639
Epoch 68/450
19/19 [==============================] - 0s 14ms/step - loss: 1.0411 - mae: 1.4658 - val_loss: 0.6534 - val_mae: 1.0663
Epoch 69/450
19/19 [==============================] - 0s 14ms/step - loss: 1.1231 

19/19 [==============================] - 0s 18ms/step - loss: 0.9234 - mae: 1.3477 - val_loss: 0.7237 - val_mae: 1.1483
Epoch 124/450
19/19 [==============================] - 0s 23ms/step - loss: 1.0286 - mae: 1.4587 - val_loss: 0.7256 - val_mae: 1.1505
Epoch 125/450
19/19 [==============================] - 0s 22ms/step - loss: 0.9313 - mae: 1.3560 - val_loss: 0.7273 - val_mae: 1.1525
Epoch 126/450
19/19 [==============================] - 0s 23ms/step - loss: 0.9635 - mae: 1.3799 - val_loss: 0.7256 - val_mae: 1.1505
Epoch 127/450
19/19 [==============================] - 1s 29ms/step - loss: 0.8502 - mae: 1.2723 - val_loss: 0.7253 - val_mae: 1.1502
Epoch 128/450
19/19 [==============================] - 0s 25ms/step - loss: 0.8734 - mae: 1.2795 - val_loss: 0.7256 - val_mae: 1.1506
Epoch 129/450
19/19 [==============================] - 0s 25ms/step - loss: 0.8899 - mae: 1.3154 - val_loss: 0.7241 - val_mae: 1.1489
Epoch 130/450
19/19 [==============================] - 0s 22ms/step - loss: 

KeyboardInterrupt: 

In [ ]:
# se multiplica por 50, 1500 epocas, 0.2 de olvidar memoria, 600 datos para entrenamiento, ventana de 2
#model.save('modelo_2_sinvarnula.h5')
# 

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [13]:
model = load_model("modelo_2_sinvarnula.h5")

ValueError: Unknown initializer: GlorotUniform

In [ ]:
y_pred=model.predict(test_X)

In [ ]:
plt.plot(test_y, color="turquoise",label="real")
plt.plot(y_pred, color="lightcoral",label="predicha")
plt.legend()
#plt.axis([200,300,-5,20])

In [ ]:
y_pred=model.predict(train_X)

In [ ]:
plt.figure(figsize=(20,15))
plt.plot(train_y, color="turquoise",label="real")
plt.plot(y_pred, color="lightcoral",label="predicha")
plt.legend()
#plt.axis([400,600,-5,10])

In [ ]:
    # make a prediction
    # yhat = model.predict(test_X)
    # test_X = test_X.reshape((test_X.shape[0], ventana*n_cols))
    # invert scaling for forecast
    # inv_yhat = concatenate((yhat, test_X[:, -19:]), axis=1)
    # inv_yhat = scaler.inverse_transform(inv_yhat)
    # inv_yhat = inv_yhat[:,0]
    # invert scaling for actual
    # test_y = test_y.reshape((len(test_y), 1))
    # inv_y = concatenate((test_y, test_X[:, -18:]), axis=1)
    # inv_y = scaler.inverse_transform(inv_y)
    # inv_y = inv_y[:,0]
